In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import sklearn
import pickle

import lime
from lime import lime_tabular
from lime import explanation


In [ ]:
data = pd.read_csv("/data.csv")
data.head()


In [ ]:
y = data["Dropout"]
X = data.drop(['Dropout'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
X_test2 = X_test
X_test_all = X_test2.join(y_test)
X_test_all = X_test_all[X_test_all["Dropout"] == 1]
X_test_all.to_csv("Test.csv")

y_testtp = X_test_all["Dropout"]
X_testtp = X_test_all.drop(['Dropout'], axis=1)

In [ ]:
explainer = lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X_train.columns,
    class_names=['no_dropout', 'dropout'],
    mode='classification'
)

In [ ]:
### Just an example - can be used with any model

mlp = MLPClassifier(random_state=1, max_iter=1000).fit(X_train, y_train)
y_pred = mlp.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
matrix = classification_report(y_test, y_pred)
print('Classification report : \n',matrix)

In [ ]:
pred_mlp = mlp.predict(X_test)

all_exp = []
for i in range(len(X_test)):
    X_t = X_test.iloc[i]
    p_t = pred_mlp[i]
    exp = explainer.explain_instance(X_t, mlp.predict_proba, num_features=10)
    a = exp.as_list()
    a2 = []
    for j in a:
        id_val = (i,)
        pt_val = (p_t,)
        s = j+id_val
        m = s+pt_val
        a2.append(m)
    all_exp.append(a2)

exp_df = pd.DataFrame(all_exp)

exp_df.to_csv("/Exp.csv")

In [ ]:
filename = '/model_mlp.sav'
pickle.dump(mlp, open(filename, 'wb'))